In [ ]:
!pip install Xgboost
import pandas as pd
from plotly import express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv("../input/taxi-pricing-with-mobility-analytics/sigma_cabs.csv")

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df["Type_of_Cab"].value_counts()

In [ ]:
df["Type_of_Cab"] = df["Type_of_Cab"].fillna("F")

In [ ]:
df["Life_Style_Index"].value_counts()

In [ ]:
df["Life_Style_Index"].describe()

In [ ]:
df["Confidence_Life_Style_Index"].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.corr()

In [ ]:
df = df.dropna(subset=["Life_Style_Index"])

In [ ]:
df.isnull().sum()

In [ ]:
sum = 0
count = 0
avg = 0
for  i in df["Customer_Since_Months"].value_counts().iteritems():
    count += i[1]
    sum += i[0]*i[1]
avg = round(sum/count,0)
df["Customer_Since_Months"] = df["Customer_Since_Months"].fillna(avg)

In [ ]:
df["Var1"] = df["Var1"].fillna(25)

In [ ]:
df.isnull().sum()

In [ ]:
def one_hot_encoding(column):
     df = pd.get_dummies(column,drop_first=True)
     return df

In [ ]:
Type_Of_Cab = one_hot_encoding(df["Type_of_Cab"])
Confidence_Life_Style_Index = one_hot_encoding(df["Confidence_Life_Style_Index"])
Destination_Type = one_hot_encoding(df["Destination_Type"])
Gender = one_hot_encoding(df["Gender"])
Type_Of_Cab = Type_Of_Cab.rename(columns={'B': 'Type_Of_Cab_B','C': 'Type_Of_Cab_C','D': 'Type_Of_Cab_D','E': 'Type_Of_Cab_E','F': 'Type_Of_Cab_F'})
Confidence_Life_Style_Index = Confidence_Life_Style_Index.rename(columns = {"B":"Confidence_Life_Style_Index_B","C":"Confidence_Life_Style_Index_C"})
Destination_Type = Destination_Type.rename(columns = {'B':'Destination_Type_B','C':'Destination_Type_C','D':'Destination_Type_D','E':'Destination_Type_E','F':'Destination_Type_F','G':'Destination_Type_G','H':'Destination_Type_H','I':'Destination_Type_I','J':'Destination_Type_J','K':'Destination_Type_K','L':'Destination_Type_L','M':'Destination_Type_M','N':'Destination_Type_N'})
print("Columns name changed Succesfully !!")

In [ ]:
df_one_hot_encoded = pd.concat([df,Type_Of_Cab,Confidence_Life_Style_Index,Destination_Type,Gender], axis=1)

In [ ]:
df_one_hot_encoded

In [ ]:
cols_to_drop = ["Trip_ID","Type_of_Cab","Confidence_Life_Style_Index","Destination_Type","Gender"]
df_final = df_one_hot_encoded.drop(cols_to_drop,axis = 1)

In [ ]:
cols_to_be_normalized = ['Trip_Distance', 'Customer_Since_Months', 'Life_Style_Index','Customer_Rating', 'Cancellation_Last_1Month', 'Var1', 'Var2', 'Var3']
cols_not_to_be_normalized = ['Type_Of_Cab_B', 'Type_Of_Cab_C', 'Type_Of_Cab_D', 'Type_Of_Cab_E','Type_Of_Cab_F', 
                            'Confidence_Life_Style_Index_B','Confidence_Life_Style_Index_C', 'Destination_Type_B',
                            'Destination_Type_C', 'Destination_Type_D', 'Destination_Type_E',
                            'Destination_Type_F', 'Destination_Type_G', 'Destination_Type_H',
                            'Destination_Type_I', 'Destination_Type_J', 'Destination_Type_K',
                            'Destination_Type_L', 'Destination_Type_M', 'Destination_Type_N',
                            'Male','Surge_Pricing_Type']

In [ ]:
normalize = normalize(df_final[cols_to_be_normalized])
normalize = pd.DataFrame(normalize,columns=cols_to_be_normalized)
binarized = df_final[cols_not_to_be_normalized].reset_index(drop = True)
df_final =  pd.concat([normalize,binarized], axis=1)

In [ ]:
df_final

In [ ]:
X = df_final.drop("Surge_Pricing_Type",axis = 1)
Y = df_final["Surge_Pricing_Type"] 

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=8,test_size=0.3,stratify=Y)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
accuracy_score(Y_test,Y_pred)

In [ ]:
print(classification_report(Y_test,Y_pred))

In [ ]:
confusion_matrix(Y_test,Y_pred)

In [ ]:
XGB = XGBClassifier()

In [ ]:
XGB.fit(X_train, Y_train)


In [ ]:
# make predictions for test data
Y_pred_new = XGB.predict(X_test)

In [ ]:
accuracy_score(Y_pred_new,Y_test)